# Part II

## Have Customers Narrow Their Travel Searches Based on Temperature and Precipitation

In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config2 import g_key

### 2. Import the WeatherPy_database.csv file from Part 1 as a new DataFrame.

In [2]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/WeatherPy_database.csv", index_col=0)
city_data_df.head()


,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Descriptions,Rain (inches last hour),Snow (inches last hour)
0,Sulam,IL,32.61,35.33,75.99,83,9,1.99,clear sky,0.00,0
1,Namatanai,PG,-3.67,152.43,86.16,65,4,5.82,clear sky,0.00,0
2,Bluff,NZ,-46.60,168.33,48.27,79,95,16.93,light rain,0.23,0
3,Santa Cruz,BO,-17.80,-63.17,59.00,93,90,1.01,overcast clouds,0.00,0
4,San Patricio,MX,19.22,-104.70,80.24,80,100,6.11,light rain,0.19,0


### 3. Filter the DataFrame

In [3]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# Ask the customer to answer if he or she would like it to be raining or not 
raining = str(input("Do you want it to be raining? (yes/no) "))
# Ask the customer to answer if he or she would like it to be snowing or not 
snowing = str(input("Do you want it to be snowing? (yes/no) "))

Do you want it to be raining? (yes/no) no
Do you want it to be snowing? (yes/no) no


In [19]:

for index, row in city_data_df.iterrows():
    if raining == "no" and snowing == "no":
        preferred_cities_df = city_data_df.loc[(city_data_df["Rain (inches last hour)"] == 0) & 
                                       (city_data_df["Snow (inches last hour)"] == 0) & 
                                       (city_data_df["Max Temp"] <= max_temp) &
                                       (city_data_df["Max Temp"] >= min_temp)]
    elif raining == "yes" and snowing == "no":
        preferred_cities_df = city_data_df.loc[(city_data_df["Rain (inches last hour)"] != 0) & 
                                       (city_data_df["Snow (inches last hour)"] == 0) & 
                                       (city_data_df["Max Temp"] <= max_temp) &
                                       (city_data_df["Max Temp"] >= min_temp)]
    elif raining == "no" and snowing == "yes":
        preferred_cities_df = city_data_df.loc[(city_data_df["Rain (inches last hour)"] == 0) & 
                                       (city_data_df["Snow (inches last hour)"] != 0) & 
                                       (city_data_df["Max Temp"] <= max_temp) &
                                       (city_data_df["Max Temp"] >= min_temp)]
    else:
        preferred_cities_df = city_data_df.loc[(city_data_df["Rain (inches last hour)"] != 0) & 
                                       (city_data_df["Snow (inches last hour)"] != 0) & 
                                       (city_data_df["Max Temp"] <= max_temp) &
                                       (city_data_df["Max Temp"] >= min_temp)]
     



In [20]:
preferred_cities_df.head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Descriptions,Rain (inches last hour),Snow (inches last hour)
0,Sulam,IL,32.61,35.33,75.99,83,9,1.99,clear sky,0.0,0
1,Namatanai,PG,-3.67,152.43,86.16,65,4,5.82,clear sky,0.0,0
15,Phuket,TH,8.00,98.25,86.00,74,20,1.12,few clouds,0.0,0
20,Aswan,EG,24.09,32.91,82.17,20,3,7.72,clear sky,0.0,0
23,Kapaa,US,22.08,-159.32,82.40,69,1,9.17,clear sky,0.0,0
26,Atuona,PF,-9.80,-139.03,80.47,76,3,14.47,clear sky,0.0,0
32,Cockburn Town,TC,21.46,-71.14,82.53,79,0,12.91,clear sky,0.0,0
33,Saskylakh,RU,71.92,114.08,88.50,25,3,11.32,clear sky,0.0,0
35,Ternate,ID,0.80,127.40,82.72,72,97,4.03,overcast clouds,0.0,0
39,Hami,CN,42.80,93.45,88.29,15,0,9.19,clear sky,0.0,0


### 4. Add the cities to a marker layer map with a pop-up marker for each city

In [21]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp","Current Descriptions", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Descriptions,Lat,Lng,Hotel Name
0,Sulam,IL,75.99,clear sky,32.61,35.33,
1,Namatanai,PG,86.16,clear sky,-3.67,152.43,
15,Phuket,TH,86.00,few clouds,8.00,98.25,
20,Aswan,EG,82.17,clear sky,24.09,32.91,
23,Kapaa,US,82.40,clear sky,22.08,-159.32,
26,Atuona,PF,80.47,clear sky,-9.80,-139.03,
32,Cockburn Town,TC,82.53,clear sky,21.46,-71.14,
33,Saskylakh,RU,88.50,clear sky,71.92,114.08,
35,Ternate,ID,82.72,overcast clouds,0.80,127.40,
39,Hami,CN,88.29,clear sky,42.80,93.45,


In [22]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [23]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [24]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Descriptions</dt><dd>{Current Descriptions}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [25]:
hotel_df

,City,Country,Max Temp,Current Descriptions,Lat,Lng,Hotel Name
0,Sulam,IL,75.99,clear sky,32.61,35.33,בורגוס בורגר בר
1,Namatanai,PG,86.16,clear sky,-3.67,152.43,Namatanai Hotel
15,Phuket,TH,86.00,few clouds,8.00,98.25,The Surin Phuket
20,Aswan,EG,82.17,clear sky,24.09,32.91,Nuba Nile
23,Kapaa,US,82.40,clear sky,22.08,-159.32,Sheraton Kauai Resort at Coconut Beach
...,...,...,...,...,...,...,...
545,Kailua,US,86.00,broken clouds,21.40,-157.74,Sheffield House Bed & Breakfast
549,Palauig,PH,85.93,overcast clouds,15.44,119.90,Myhousebythesea
558,Xianyang,CN,80.01,mist,34.34,108.70,铁路大厦
565,Umm Lajj,SA,82.83,few clouds,25.02,37.27,Moon light Furnished Units


In [26]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig


Figure(layout=FigureLayout(height='420px'))

In [27]:
# Set the destination of the output file (CSV).
output_data_file = "weather_data/WeatherPy_vacation.csv"
city_data_df.to_csv(output_data_file, index= False)